<a href="https://colab.research.google.com/github/kapamawi/AI/blob/main/2_1_6_ollama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Przygotowanie

1. instalacja Ollamy https://ollama.com/
2. z konsoli:  ollama pull llama3.1:latest

## Skrypt

Kod poniżej --> zapisać do pliku app.py

In [ ]:
import ollama
import requests

class CFG:
    model = 'llama3.1:latest'

def get_current_weather(city):
    base_url = f"https://wttr.in/{city}?format=j1"
    response = requests.get(base_url)
    data = response.json()
    return f"Temp in {city}: {data['current_condition'][0]['temp_C']}"


def what_is_bigger(n, m):
    if n > m:
        return f"{n} is bigger"
    elif m > n:
        return f"{m} is bigger"
    else:
        return f"{n} and {m} are equal"


def chat_with_ollama_no_functions(user_question):
    response = ollama.chat(
        model = CFG.model,
        messages=[
            {'role': 'user', 'content': user_question}
        ]
    )
    return response


def chat_with_ollama(user_question):
    response = ollama.chat(
        model = CFG.model,
        messages=[
            {'role': 'user', 'content': user_question}
        ],
        tools=[
            {
                'type': 'function',
                'function': {
                    'name': "get_current_weather",
                    'description': "Get the current weather for a city",
                    'parameters': {
                        'type': "object",
                        'properties': {
                            'city': {
                                'type': "string",
                                "description": "City",
                            },
                        },
                        'required': ['city'],
                    },
                },
            },
            {
                'type': "function",
                'function': {
                    "name": "which_is_bigger",
                    'parameters': {
                        'type': 'object',
                        'properties': {
                            'n': {
                                'type': "float",
                            },
                            "m": {
                                'type': "float"
                            },
                        },
                        'required': ['n', 'm'],
                    },
                },
            },
        ],
    )
    return response


def main():
    while True:
        user_input = input("Enter your question (or 'quit' to exit): ")
        if user_input.lower() == 'quit':
            break

        response = chat_with_ollama(user_input)

        if 'tool_calls' in response['message'] and response['message']['tool_calls']:
            tools_calls = response['message']['tool_calls']
            for tool_call in tools_calls:
                tool_name = tool_call['function']['name']
                arguments = tool_call['function']['arguments']

                if tool_name == 'get_current_weather' and 'city' in arguments:
                    result = get_current_weather(arguments['city'])
                    print("Weather function result:", result)
                elif tool_name == 'which_is_bigger' and 'n' in arguments and 'm' in arguments:
                    n, m = float(arguments['n']), float(arguments['m'])
                    result = what_is_bigger(n, m)
                    print("Comparison function result:", result)
                else:
                    print(f"No valid arguments found for function: {tool_name}")
        else:
            # If no tool calls or no valid arguments, use the LLM's response
            response = chat_with_ollama_no_functions(user_input)
            print("AI response:", response['message']['content'])


if __name__ == "__main__":
    main()


Kod tworzy interaktywny system czatu wykorzystujący model językowy Ollama z dodatkowymi funkcjami. Przeanalizujmy jego główne elementy:

Funkcja `get_current_weather(city)` pobiera aktualną pogodę dla wskazanego miasta:
- Łączy się z serwisem wttr.in
- Zwraca temperaturę w stopniach Celsjusza dla podanego miasta

Funkcja `what_is_bigger(n, m)` porównuje dwie liczby:
- Sprawdza która z liczb n i m jest większa
- Zwraca odpowiedni komunikat tekstowy z wynikiem porównania

Funkcja `chat_with_ollama_no_functions(user_question)` realizuje prostą interakcję z modelem Ollama:
- Używa modelu llama3.1:8b-instruct-fp16
- Przyjmuje pytanie użytkownika i zwraca odpowiedź modelu

Funkcja `chat_with_ollama(user_question)` to rozszerzona wersja czatu:
- Wykorzystuje model llama3.1:latest
- Ma dostęp do dwóch dodatkowych funkcji (narzędzi):
  1. Sprawdzanie pogody
  2. Porównywanie liczb
- Definiuje strukturę i parametry tych funkcji

Funkcja `main()` zarządza interakcją z użytkownikiem:
- Działa w pętli do momentu wpisania "quit"
- Przetwarza wprowadzone pytania
- Jeśli model zdecyduje użyć jednej z funkcji:
  - Wywołuje odpowiednią funkcję z przekazanymi argumentami
  - Wyświetla wynik
- W przeciwnym razie:
  - Używa podstawowej wersji czatu
  - Wyświetla bezpośrednią odpowiedź modelu

Kod wykorzystuje biblioteki:
- ollama - do komunikacji z modelem językowym
- requests - do pobierania danych pogodowych